In [323]:
import gym
import numpy as np
import copy

I started by using the Frozen lake environment in the gym distribution, after that I felt like upscaling it. So I jumped to the Taxi-v3 environment. I have implemented a Markov Decision Process, in which we use policy iteration to find the optimal policy. 

The rules of the "Taxi-v3" environment is really interesting. There are four location on the city marked by R,G,B and Y. These are passenger pick up and drop off locations.

The rewards are as follows, for a sucessfull pick-up and drop off the driver gets 20 points, for a illegal pickup or dropoff the driver gets penalized by 10 points, for every step that the driver takes 1 point is reduced (this motivates the driver to take the optimal path between pick up and dropoff).

There are a total of 500 states in this environment. The value for each state are calculated and Optimal policy is found by policy iteration. 

In [324]:
# loading the environment. 
# In the render, the yellow brick is the taxi, blue color is the passenger location and Magenta is the drop-off location. 
env = gym.make("Taxi-v3")  
env.reset()   
env.render() 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [333]:
# initiating a random policy.
def rand_policy(env):
    policy = np.empty([env.nS,env.nA])
    for i in range(env.nS):
        item = np.random.randint(1,3,env.nA)
        policy[i] = item/sum(item)
    return policy


# env: is the environment
# policy: is the policy that we use to eveluate the values. This is a probability matrix of actions for each state. 
# discount: is the discount factor

def policy_eval(env, policy, discount):
    # initializing a values vector, with size # of states
    values = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            v = 0   
            for a, prob_a in enumerate(policy[s]): # a is the action, prob_a is the probability of the action.
                for probability, next_state, reward, done in env.P[s][a]:
                    v += prob_a * probability * (reward + discount * values[next_state])
            delta = max(delta, np.abs(v - values[s]))
            values[s]= v
        #  Checking if the values have sufficiently converged 
        if delta < 1e-8:
            break
    # expected value of state s under the input policy.
    return values




In [326]:
#Finding the expected values for actions from a state 's'.
#Parameters.
#env: environment
#v: values of states
#s: current state
#discount: discount factor

def value_state(env,v,s,discount):
    q = np.zeros(env.nA)
    for a in range(env.nA):
        for probability, next_state, reward, done in env.P[s][a]:
            q[a] += probability * (reward + discount * v[next_state])
    return q

# Improving policy using the Values of states. 
def policy_improv(env,v,discount):
    policy = np.zeros([env.nS,env.nA])
    for s in range(env.nS):
        q = value_state(env,v,s,discount)
        policy[s][np.argmax(q)] = 1
    return policy


def policy_iteration(env, discount):
    policy = rand_policy(env) # random initial policy
    iter = 0
    while True:
        values = policy_eval(env, policy, discount) # finding value of states under that policy
        new_policy = policy_improv(env, values, discount) # improving the policy
        
        # Checking for convergence of policy
        if (new_policy == policy).all():
            print("Policy convergency achieved! Convergence was achieved in ", iter, "iterations")
            break

        policy = copy.copy(new_policy)
        iter += 1
    return policy


We find the optimal policy. 

In [334]:
 opt_policy = policy_iteration(env,0.9)

Policy convergency achieved! Convergence was achieved in  14 iterations


This is where we test the policy we got and see if the taxi driver is taking appropriate paths after training with the rewards. 

First we show what the taxi does with a random policy and also calculate the rewards it accumilated. We only do 50 iterations. 

In [335]:
state = env.reset()
rewards = 0
policy = rand_policy(env)
for step in range(50):
        env.render()
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(policy[state])        
        new_state, reward, done, info = env.step(action)
        rewards += reward
        state = new_state
        if done or step == 49:
            print ("Cumulative reward for this episode so far is  ",rewards)
            break

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:

This is how the driver performs under the optimal policy!
The results are impressive and easily visualized. 

In [336]:
# We examine 5 different senarios and also calculate the reward accumilated by the driver.
# Max reward that can be achieved is 20.(The actual maximum is less that this! 20 was an easy upper bound.)
for episode in range(5):
    state = env.reset()
    rewards = 0
    print("****************************************************")
    print("EPISODE ", episode) 
    print("Yellow Brick - Taxi, Green Brick - Taxi with passenger")

    for step in range(100):
        env.render()
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(opt_policy[state])        
        new_state, reward, done, info = env.step(action)
        rewards += reward
        state = new_state
        if done:
            print ("Cumulative reward for this episode is ", rewards)
            break

****************************************************
EPISODE  0
Yellow Brick - Taxi, Green Brick - Taxi with passenger
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
|_: : : :